In [1]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn #neural network module
import torch.optim as optim #optimizer
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import transforms

import time as time
from tqdm import tqdm

import pickle
import gc
import os

In [2]:
datasetpath=os.path.realpath('MonophonicDataset.pt')
PolDataset = os.path.realpath('PolyphonicDataset.pt')

Mono = False

In [7]:
#Loading monophonic and polyphonic classes
class MonophonicDataset(Dataset):

   def __init__(self, Instruments):

      DS = torch.load(datasetpath)
      self.Data = []
      self.Instruments = Instruments

      for inst in Instruments:

        self.Data.extend(DS[inst])

      del DS
      gc.collect()


   def __len__(self):
      return len(self.Data)

   def __getitem__(self, idx):

      PreviousBars = self.Data[idx]['Bars'][0].to_dense()
      Bars = self.Data[idx]['Bars'][1].to_dense()

      prog = self.Data[idx]['Program']
      tempo = self.Data[idx]['Tempo'][0]

      TEMPO_MIN, TEMPO_MAX = 60, 200
      PROGRAM_MIN, PROGRAM_MAX = 1, 128

      tempo_norm = (tempo - TEMPO_MIN) / (TEMPO_MAX - TEMPO_MIN)
      prog_norm = (prog - PROGRAM_MIN) / (PROGRAM_MAX - PROGRAM_MIN)


      Cond1D = torch.tensor([tempo_norm] + [prog_norm], dtype=torch.float, device=Bars.device)
      return Bars, PreviousBars, Cond1D




class PolyphonicDataset(Dataset):

   def __init__(self):

         DS = torch.load(PolDataset, weights_only=False)
         self.Data = []

         self.Data.extend(DS)

         del DS
         gc.collect()

   def __len__(self):
      return len(self.Data)

   def __getitem__(self, idx):

      PreviousBars = self.Data[idx]['Bars'][0].to_dense()
      Bars = self.Data[idx]['Bars'][1].to_dense()

      prog = self.Data[idx]['Program']
      tempo = self.Data[idx]['Tempo']
      genre = self.Data[idx]['Genre']


      TEMPO_MIN, TEMPO_MAX = 60, 200
      PROGRAM_MIN, PROGRAM_MAX = 1, 130
      GENRE_MIN, GENRE_MAX = 0, 9

      tempo_norm = (tempo - TEMPO_MIN) / (TEMPO_MAX - TEMPO_MIN)
      genre_norm = (genre - GENRE_MIN) / (GENRE_MAX - GENRE_MIN)
      prog_norm = [(p - PROGRAM_MIN) / (PROGRAM_MAX - PROGRAM_MIN) for p in prog]


      Cond1D = torch.tensor([tempo_norm, genre_norm] + prog_norm, dtype=torch.float, device=Bars.device)
      return Bars, PreviousBars, Cond1D

TypeError: list expected at most 1 argument, got 3

In [21]:
EPOCHS = 20
noise_dim = 100
BATCH_SIZE = 72

if Mono: Data = MonophonicDataset(Instruments=['Piano'])
else: Data = PolyphonicDataset()

def getDataloader(dataset, batch_size, num_batches):
    # Total samples to use per epoch
    subset_size = batch_size * num_batches
    indices = np.random.choice(len(dataset), size=subset_size, replace=False)
    sampler = SubsetRandomSampler(indices)
    return DataLoader(dataset, batch_size=batch_size, sampler=sampler, drop_last=True)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('GPU available')

elif torch.backends.mps.is_available():
    device = torch.device('mps')
    print('mps available')
else:
    device = torch.device("cpu")
    print('GPU not available')

### Concatenation

Definition of the concatenation functions that are used in the hidden layers to concatenate the output and the 1_d and 2_d conditions.

1_d conditioning vector of shape $[n,1]$ with an output of shape $[batch,features,a,b]$:
* first we have to duplicate the vector $a\cdot b$ times to get a tensor of shape $[batch,n,a,b]$
* then we can concatenate the two tensors in the depth dimension (i.e dim=1)

2_d conditioning matrix of the same shape of the output $[batch,features,a,b]$ except the depth dim (it must be that because how we build the conditioner CNN):
* first we check that the dimensions are correct
* we concatenate the two tensors in the depth dimension (i.e dim=1)

In [8]:
def conv_cond_concat(x, y):
    """Concatenate conditioning vector on feature map axis."""
    x_shapes = x.shape  #[batch,n_features,a,b]
    y_shapes = y.shape  #[batch,n]
    y2 = y.view(x_shapes[0],y_shapes[1],1,1)                              #[batch,n,1,1]
    y2 = y2.expand(x_shapes[0],y_shapes[1],x_shapes[2],x_shapes[3])      #[batch,n,a,b]

    return torch.cat((x, y2),dim=1)                                     #[batch,n_features+n,a,b]

def conv_prev_concat(x, y):
    """Concatenate conditioning vector on feature map axis."""
    x_shapes = x.shape  #[batch,n_features,a,b]
    y_shapes = y.shape  #[batch,16,a,b]
    if x_shapes[2:] == y_shapes[2:]:
        y2 = y.expand(x_shapes[0],y_shapes[1],x_shapes[2],x_shapes[3])  #[batch,16,a,b]

        return torch.cat((x, y2),dim=1)                                 #[batch,n_features+16,a,b]

    else:
        print(x_shapes[2:])
        print(y_shapes[2:])

### The Generator and the Conditioner

The generator uses `ConvTranspose2d` (upsampling) layers to produce an image from a seed (random noise). Start with two `Dense` layers that take this seed as input and transform it to a tensor of shape $[batch size,128,1,2]$, then upsample several times until we reach the desired size of a bar of $[instrument,128,16]$. We use  the `ReLU` activation for each layer, except the output layer which can use `Sigmoid` to predict pixel values in the [0, 1] range.

Coupled to the generator there is the conditioner that uses `Conv2d` (sampling) layers to produce the 2_d tensors that serve as informations from the preaviou bar. The conditioner can be viewed as the reverse of the generator because it uses filters with the same shapes of the ones in the generator. In this case we use  the `LeakyReLU` activation for each layer.

In [9]:
class Generator(nn.Module):

    def __init__(self, input_size, cond_1d_size, instrument_size=1, n_hlayers=128):
            super().__init__()

            self.input_size = input_size
            self.instrument_size = instrument_size
            self.cond1d_dim = cond_1d_size

            #generator layers
            #as said in the DCGAN paper always ReLU activation function in the generator excluded the last layer
            #as said in the DCGAN paper always batchnorm iin the generator excluded the last layer
            self.ff1 = nn.Sequential(
                nn.Linear(input_size+cond_1d_size, 1024),                                                               #[batch,1024]
                nn.BatchNorm1d(1024),
                nn.ReLU()
                )
            self.ff2 = nn.Sequential(
                nn.Linear(1024+cond_1d_size,n_hlayers*2),                                                                                    #[batch,512]
                nn.BatchNorm1d(n_hlayers*2),
                nn.ReLU()
                )
            #reshape to [batch size,128,1,2]
            # #+condition [batch,128+cond_1d_size+16,1,2]
            self.cnn1 = nn.Sequential(
                nn.ConvTranspose2d(n_hlayers+cond_1d_size+16, n_hlayers, kernel_size=(1,2), stride=(2,2), bias=False, padding=0),           #[batch,128,1,4]
                nn.BatchNorm2d(n_hlayers),
                nn.ReLU()
                )
            #+condition [batch,128+cond_1d_size+16,1,2]
            self.cnn2 = nn.Sequential(
                nn.ConvTranspose2d(n_hlayers+cond_1d_size+16, n_hlayers, kernel_size=(1,2), stride=(2,2), bias=False, padding=0),           #[batch,128,1,8]
                nn.BatchNorm2d(n_hlayers),
                nn.ReLU()
                )
            #+condition [batch,128+cond_1d_size+16,1,2]
            self.cnn3 = nn.Sequential(
                nn.ConvTranspose2d(n_hlayers+cond_1d_size+16, n_hlayers, kernel_size=(1,2), stride=(2,2), bias=False, padding=0),           #[batch,128,1,16]
                nn.BatchNorm2d(n_hlayers),
                nn.ReLU()
                )
            #+condition [batch,128+cond_1d_size+16,1,2]
            self.cnn4 = nn.Sequential(
                nn.ConvTranspose2d(n_hlayers+cond_1d_size+16, instrument_size, kernel_size=(128,1), stride=(2,1), bias=False, padding=0),       #[batch,instrument_size,128,16]
                nn.Sigmoid()
                #Sigmoid funciotn because we want to generate the matrixes of music without velocity, i.e. only (0,1)
                #Thus we use the sigmoid which is a smoother version of the sign function
                )
            #conditioner layers
            # #as in Midinet model we use the Leaky activation funciton for the conditioner
            self.h0_prev = nn.Sequential(
                nn.Conv2d(in_channels=instrument_size, out_channels=16, kernel_size=(128,1), stride=(2,1)),                  #[batch,16,1,16]
                nn.BatchNorm2d(16),
                nn.LeakyReLU()          #note: in the original paper leak=0.2, default leak=0.01
                )
            self.h1_prev = nn.Sequential(
                nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(1,2), stride=(2,2)),                                  #[batch,16,1,8]
                nn.BatchNorm2d(16),
                nn.LeakyReLU()
                )
            self.h2_prev = nn.Sequential(
                nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(1,2), stride=(2,2)),                                  #[batch,16,1,4]
                nn.BatchNorm2d(16),
                nn.LeakyReLU()
                )
            self.h3_prev = nn.Sequential(
                nn.Conv2d(in_channels=16, out_channels=16, kernel_size=(1,2), stride=(2,2)),                                  #[batch,16,1,2]
                nn.BatchNorm2d(16),
                nn.LeakyReLU()
                )

    def forward(self, z, prev_bar, cond_1d, batch_size):

            #2d condiiton
            cond0 = self.h0_prev(prev_bar)          #[batch,16,1,16]
            cond1 = self.h1_prev(cond0)             #[batch,16,1,8]
            cond2 = self.h2_prev(cond1)             #[batch,16,1,4]
            cond3 = self.h3_prev(cond2)             #[batch,16,1,2]

            #single cond_1d size =[n,1], batch_cond_1d size = [batch_size,n]

            input = torch.cat((z,cond_1d), dim=1)   #[batch_size, input_size+cond_1d_size]

            h0 = self.ff1(input)                    #[batch,1024]
            h0 = torch.cat((h0,cond_1d), dim=1)     #[batch,1024+cond_1d_size]

            h1 = self.ff2(h0)                       #[batch,256]
            h1 = h1.reshape(batch_size, 128, 1, 2)  #[batch,128,1,2]
            h1 = conv_cond_concat(h1,cond_1d)       #[batch,128+cond_1d_size,1,2]
            h1 = conv_prev_concat(h1,cond3)         #[batch,128+cond_1d_size+16,1,2]

            h2 = self.cnn1(h1)                      #[batch,128,1,4]
            h2 = conv_cond_concat(h2,cond_1d)       #[batch,128+cond_1d_size,1,4]
            h2 = conv_prev_concat(h2,cond2)         #[batch,128+cond_1d_size+16,1,4]

            h3 = self.cnn2(h2)                      #[batch,128,1,8]
            h3 = conv_cond_concat(h3,cond_1d)       #[batch,128+cond_1d_size,1,8]
            h3 = conv_prev_concat(h3,cond1)         #[batch,128+cond_1d_size+16,1,8]

            h4 = self.cnn3(h3)                      #[batch,128,1,16]
            h4 = conv_cond_concat(h4,cond_1d)       #[batch,128+cond_1d_size,1,16]
            h4 = conv_prev_concat(h4,cond0)         #[batch,128+cond_1d_size+16,1,16]

            out = self.cnn4(h4)             #[batch,instrument_size,128,16]

            return out

### The Discriminator

The discriminator uses `Conv2d` (sampling) layers to produce a scalar output from a bar input. Start with two `Conv2d` layers that reduce the size of the input, then use two `Dense` layers. We use  the `LeakyReLU` activation for each layer, except the output layer which can use `Sigmoid` to predict true-false probability value in the [0, 1] range. Note that the activation is included in the loss function.

In [10]:
class MinibatchDiscrimination(nn.Module):
    def __init__(self, in_features, out_features, kernel_dim):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.kernel_dim = kernel_dim
        self.T = nn.Parameter(torch.randn(in_features, out_features, kernel_dim))

    def forward(self, x):
        # x is batch_sizexA
        # T is AxBxC
        matrices = x @ self.T.view(self.in_features, -1)                # matrix moltiplication, result shape: [batch_size, B*C]
        matrices = matrices.view(-1, self.out_features, self.kernel_dim)    #M shape [batch, B, C]

        # compute L1 distances between samples
        M = matrices.unsqueeze(0)  # [1,batch,B,C]
        M_T = M.permute(1, 0, 2, 3)  # [batch,1,B,C]
        norm = torch.abs(M - M_T).sum(3)  # first broadcast [batch,batch,B,C], then [batch,batch,B]
        cbij = torch.exp(-norm)
        o_b = cbij.sum(0)   # [batch,B], if j !=0 i in teh sum then subtract self distance (cbij.sum(0) - 1)

        x = torch.cat([x, o_b], 1)
        return x

In [11]:
class Discriminator(nn.Module):

    def __init__(self, cond_1d_size, instrument_size=1, mini_size=100):
        super().__init__()

        self.instrument_size = instrument_size
        self.cond1d_dim = cond_1d_size

        #as said in the DCGAN paper always batchnorm in the discriminator layers excluded the first layer
        self.cnn1 = nn.Sequential(
            nn.Conv2d(2*instrument_size+cond_1d_size, 32, kernel_size=(128,2), stride=(2,2), padding=0),        #[batch,32,1,8]
            nn.LeakyReLU(0.2),
            nn.Dropout(0.1)
        )
        #+condition [batch,14+cond_1d_size,1,8]
        self.cnn2 = nn.Sequential(
            nn.Conv2d(32+cond_1d_size, 77, kernel_size=(1,4), stride=2, padding=0),                             #[batch,77,1,3]
            #Adding residual block
            nn.LeakyReLU(0.2),
            nn.Dropout(0.1)
        )

        self.ffnn1 = nn.Sequential(
             #+condition [batch,231+cond_1d_size]
            nn.Linear(231+cond_1d_size, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2)
        )

        self.minibatch = MinibatchDiscrimination(in_features=1024, out_features=mini_size,kernel_dim=3)

        #+condition [batch,1024+mini_size+cond_1d_size]
        self.ffnn2 = nn.Linear(1024+cond_1d_size+mini_size, 1)      #no sigmoid activation function because it is already in the definition of the cross entropy loss function


    def forward(self, x, prev_bar, cond_1d):
        input = conv_cond_concat(x,cond_1d)         #[batch,instrument_size+cond_1d_size,128,16]
        input = conv_prev_concat(input,prev_bar)    #[batch,2*instrument_size+cond_1d_size,128,16]

        h0 = self.cnn1(input)                       #[batch,14,1,8]
        fm = h0
        h0 = conv_cond_concat(h0, cond_1d)          #[batch,14+cond_1d_size,1,8]

        h1 = self.cnn2(h0)                          #[batch,77,1,3]
        h1 = torch.flatten(h1, 1)                   #[batch,77*3*1]
        h1 = torch.cat((h1,cond_1d),dim=1)          #[batch,231+cond_1d_size]

        h2 = self.ffnn1(h1)                         #[batch,1024]
        h2 = self.minibatch(h2)                     #[batch,1024+mini_size]
        h2 = torch.cat((h2,cond_1d),dim=1)          #[batch,1024+mini_size+cond_1d_size]

        h3 = self.ffnn2(h2)                         #[batch,1]
        h3_sigmoid = torch.sigmoid(h3)


        return h3_sigmoid, h3, fm



### Weights initialization

Is this ok?

In [12]:
# def weights_init(m):
#     classname = m.__class__.__name__
#     if classname.find('Conv') != -1:
#         nn.init.xavier_uniform_(m.weight.data)*0
#     if classname.find('Linear') != -1:
#         nn.init.xavier_uniform_(m.weight.data)*0
#     elif classname.find('BatchNorm') != -1:
#         nn.init.normal_(m.weight.data, 1.0, 0.2)*0
#         nn.init.constant_(m.bias.data, 0)*0


def weights_init(m):
    if isinstance(m, (nn.ConvTranspose2d, nn.Conv2d)):
        nn.init.normal_(m.weight.data, 0.0, 0.02)  # DCGAN standard
    elif isinstance(m, nn.Linear):
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d)):
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


### Creation of the model

In [13]:
if Mono:
  generator = Generator(input_size=100, cond_1d_size=2, instrument_size=1, n_hlayers=128)
else:
  generator = Generator(input_size=100, cond_1d_size=6, instrument_size=4, n_hlayers=128)
generator.apply(weights_init)

Generator(
  (ff1): Sequential(
    (0): Linear(in_features=106, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (ff2): Sequential(
    (0): Linear(in_features=1030, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (cnn1): Sequential(
    (0): ConvTranspose2d(150, 128, kernel_size=(1, 2), stride=(2, 2), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (cnn2): Sequential(
    (0): ConvTranspose2d(150, 128, kernel_size=(1, 2), stride=(2, 2), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (cnn3): Sequential(
    (0): ConvTranspose2d(150, 128, kernel_size=(1, 2), stride=(2, 2), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, trac

In [14]:
if Mono:
  discriminator = Discriminator(cond_1d_size=2, instrument_size=1)
else:
  discriminator = Discriminator(cond_1d_size=6, instrument_size=4)
discriminator.apply(weights_init)

Discriminator(
  (cnn1): Sequential(
    (0): Conv2d(14, 32, kernel_size=(128, 2), stride=(2, 2))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.1, inplace=False)
  )
  (cnn2): Sequential(
    (0): Conv2d(38, 77, kernel_size=(1, 4), stride=(2, 2))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.1, inplace=False)
  )
  (ffnn1): Sequential(
    (0): Linear(in_features=237, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
    (3): Dropout(p=0.2, inplace=False)
  )
  (minibatch): MinibatchDiscrimination()
  (ffnn2): Linear(in_features=1130, out_features=1, bias=True)
)

### Dimension testing

Produce a noise vector of size `[10, 100]`, a noise 1d condition vector of size `[10, 15]`, and a noise 2d condition tensor of size `[10, 1, 128,16]`. Note that we need a 1d and a 2d contions for each batch input. Then we use the (as yet **untrained**) generator to create an image of expected output shape $[10,1,128,16]$.

Then use the (yet **untrained**) discriminator to classify the generated images as real or fake. The model will be trained to output the probability that the image is real in the first output component, thus we expect an output vector of size `[10, 1]` with $x_i \in [0,1]$.

In [15]:
############################ input (batch_size=10, vector_size=100) ###############################
noise = torch.normal(0, 1, [10, 100])
print(noise.shape)
############################ conditions ###############################
if Mono:
  cond_1d = torch.normal(0,1,[10,2])
  prev_bar = torch.normal(0, 1, [10, 1, 128, 16])
else:
  cond_1d = torch.normal(0,1,[10,6])
  prev_bar = torch.normal(0, 1, [10, 4, 128, 16])


############################ generator ###############################
generated_bar = generator(noise, prev_bar, cond_1d, batch_size=10).detach()
print(generated_bar.shape)
############################ discriminator ###############################
decision, __, __= discriminator(generated_bar, prev_bar, cond_1d)
print(decision)

torch.Size([10, 100])
torch.Size([10, 4, 128, 16])
tensor([[0.5908],
        [0.6582],
        [0.3681],
        [0.4791],
        [0.3525],
        [0.6121],
        [0.1656],
        [0.6174],
        [0.4862],
        [0.4991]], grad_fn=<SigmoidBackward0>)


In [16]:
generated_bar[0][0]

tensor([[0.4541, 0.5012, 0.5011,  ..., 0.4202, 0.4675, 0.4239],
        [0.4139, 0.4806, 0.3858,  ..., 0.4001, 0.4306, 0.4693],
        [0.5372, 0.5582, 0.5603,  ..., 0.5381, 0.6020, 0.5458],
        ...,
        [0.4588, 0.4646, 0.5102,  ..., 0.4447, 0.4528, 0.4843],
        [0.5228, 0.5080, 0.4965,  ..., 0.6051, 0.5192, 0.5094],
        [0.4600, 0.4283, 0.5494,  ..., 0.5242, 0.4691, 0.5199]])

### Discriminator loss

This method quantifies how well the discriminator is able to distinguish real images from fakes. It compares the discriminator's predictions on real images to an array of 1s, and the discriminator's predictions on fake (generated) images to an array of 0s.
The discriminator loss is of the form:

$\frac{1}{m}\sum_{i=1}^{m}-[\log D(\boldsymbol{x}^{(i)}) +\log(1-D(G(\boldsymbol{z}^{(i)})))]$

We inplement one-sided label smoothing to penalize self confidence and imporve the convergence of the training. Thus we substitute the discriminator's predictions on real images to an array of 1s with an array of (1-$\alpha$)s and the loss function becomes:

$\frac{1}{m}\sum_{i=1}^{m}-[(1-\alpha) \log D(\boldsymbol{x}^{(i)}) +\alpha \log (1-D(\boldsymbol{x}^{(i)}))+\log(1-D(G(\boldsymbol{z}^{(i)})))]$

In [17]:
cross_entropy = nn.BCEWithLogitsLoss()
MSE=nn.MSELoss()

In [18]:
def discriminator_loss(real_output, fake_output, device, alpha=0.1):

    #real_targets = torch.ones_like(real_output, device=device)                 #no label smoothing -> True output expected output is 1
    real_targets = torch.full_like(real_output, 1.0 - alpha, device=device)     #one side label smoothing to penalize self confidence
    fake_targets = torch.zeros_like(fake_output, device=device)                 #no label smoothing -> Fake output expected output is 0

    real_loss = cross_entropy(real_output, real_targets)
    fake_loss = cross_entropy(fake_output, fake_targets)

    total_loss = real_loss + fake_loss
    return total_loss

### Generator loss

The generator's loss quantifies how well it was able to trick the discriminator. Intuitively, if the generator is performing well, the discriminator will classify the fake images as real (or 1).
The generator loss is of the form:

$\frac{1}{m}\sum_{i=1}^{m}\log(1-D(G(\boldsymbol{z}^{(i)})))$

However this loss has some convergence issues due to vanishing gradients. So instead we use the following loss which has the same trend but stronger gradient when the discriminator is too good at recognizing fake samples.

$\frac{1}{m}\sum_{i=1}^{m}-\log(D(G(\boldsymbol{z}^{(i)})))$

Which is the Binary crossentropy between $D(G(\boldsymbol{z}^{(i)}))$ and the probability distribution that has $y^{(i)} = 1 \forall i$, i.e. we are forcing the generator to produce samples that will make the discriminator predict that fake samples are real.

Moreover we add a regularizer term so-called feature matching such that the distributions of the real and generated data are enforced to be close.

$\lambda_1 ||E_{x \sim p(x)} [x] - E_{z\sim p(z)} [G(z)] ||^2 + \lambda_2 ||E_{x \sim p(x)} [f(x)] - E_{z \sim p(z)} [f(G(z))] ||^2$


In [19]:
def generator_loss(fake_output, real_bar, fake_bar, real_f, fake_f, device, lambda1=0.1, lambda2=0.01):

    gen_loss = cross_entropy(fake_output, torch.ones_like(fake_output, device=device))

    mean_real = torch.mean(real_bar, dim=0)
    mean_fake = torch.mean(fake_bar, dim=0)
    l2_data = MSE(mean_real, mean_fake)

    mean_real_feat = torch.mean(real_f, dim=0)
    mean_fake_feat = torch.mean(fake_f, dim=0)
    l2_feat = MSE(mean_real_feat, mean_fake_feat)

    return gen_loss+lambda1*l2_data+lambda2*l2_feat

### Optimizers

With DCGAN the training is very diffuclt so we decide to use Adam optimizer as suggested by the paper. Note that with Adam we use both momentum and RMSprop to normalized velocities. Discriminator and generator need two different optimizers (conditioner is included in the generator training).

In [20]:
generator.to(device)
discriminator.to(device)

gen_opt = optim.Adam(generator.parameters(), lr=1e-4, betas=(0.5, 0.999))
dis_opt = optim.Adam(discriminator.parameters(), lr=4e-4, betas=(0.5, 0.999))

### Training loop

In [21]:
def LossPenalty(GenBars, ActualLoss):

   mean = GenBars.mean().item()
   BinaryBar = (GenBars > mean).float().squeeze(0).squeeze(0).cpu().numpy()

   ActiveNotes = np.sum(BinaryBar, axis = 0)

   Notes = (ActiveNotes > 5)
   Sums = np.sum(Notes)

   #PenaltyTerm on the loss if more that 5 notes are active at the same time
   PenaltyTerm = ActualLoss + 0.05*ActualLoss*Sums

   return PenaltyTerm

In [22]:
def train_step(bars, cond_1d, prev_bar, generator, discriminator, batch_size,
               noise_dim, device, dis_opt, gen_opt):
    # --- Ensure all tensors are on the correct device ---
    bars = bars.to(device)
    cond_1d = cond_1d.to(device)
    prev_bar = prev_bar.to(device)

    # --- Discriminator training ---
    noise = torch.randn([batch_size, noise_dim], device=device)

    bars_noise = bars + torch.randn_like(bars) * 0.1
    bars_noise = torch.clamp(bars_noise, 0, 1)

    # Generate fake samples
    generated_bars = generator(noise, prev_bar, cond_1d, batch_size)

    # Forward pass (real + fake)
    real_output, real_D, _ = discriminator(bars_noise, prev_bar, cond_1d)
    fake_output, fake_D, _ = discriminator(generated_bars.detach(), prev_bar, cond_1d)

    # Discriminator loss
    disc_loss = discriminator_loss(real_D, fake_D, device)
    discriminator.zero_grad()
    disc_loss.backward()
    dis_opt.step()

    # --- Generator training (2 steps) ---
    gen_losses = []
    for _ in range(1):  # Consistent 2:1 update ratio
        noise = torch.randn([batch_size, noise_dim], device=device)
        generated_bars = generator(noise, prev_bar, cond_1d, batch_size)
        _, fake_D, fake_fm = discriminator(generated_bars, prev_bar, cond_1d)

        with torch.no_grad():
            _, real_D, real_fm = discriminator(bars_noise, prev_bar, cond_1d)

        gen_loss = generator_loss(fake_D, bars, generated_bars, real_fm, fake_fm, device)

        generator.zero_grad()
        gen_loss.backward()
        gen_opt.step()
        gen_losses.append(gen_loss.item())

    return sum(gen_losses) / len(gen_losses), disc_loss.item()

supponendo che nel dataloader ogni dato sia una bar + la preavious bar + 1d condition sugli strumenti utilizzati

In [23]:
# gloss = []
# dloss = []

# for epoch in range(EPOCHS):

#     start = time.time()
#     generator.train()
#     discriminator.train()
#     gen_losses = []
#     disc_losses = []
#     print('#################')
#     print(f"Epoch: {epoch+1}")

#     iterator = tqdm(dataloader)
#     for bar_batch, prev_bar_batch, instrument_batch in iterator:
#         bar_batch = bar_batch.to(dtype=torch.float32, device=device)
#         prev_bar_batch = prev_bar_batch.to(dtype=torch.float32, device=device)
#         instrument_batch = instrument_batch.to(dtype=torch.float32, device=device)
#         #instrument_batch = torch.zeros_like(cond_1d)

#         if Mono:
#             bar_batch = bar_batch.unsqueeze(1)
#             prev_bar_batch=prev_bar_batch.unsqueeze(1)

#         gen_loss, disc_loss = train_step(bar_batch, instrument_batch, prev_bar_batch, generator, discriminator,
#                                          BATCH_SIZE, noise_dim, device, dis_opt, gen_opt)
#         gen_losses.append(gen_loss)
#         disc_losses.append(disc_loss)

#         iterator.set_description('Discriminator loss: {}, Generator loss: {}'.format(disc_loss, gen_loss))

#     gloss.append(np.mean(gen_losses))
#     dloss.append(np.mean(disc_losses))
#     #print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
#     print(f'dLoss: {dloss[-1]}, gLoss: {gloss[-1]}')


### Weights and loss analysis

First let's plot the 2 losses over the epochs, if it works correctly the generator loss would have to decrease and the discriminator one would have to increase

In [52]:
plt.plot(gloss, label='Generator loss')
plt.plot(dloss, label='Discriminator loss')
plt.legend()
plt.show()

NameError: name 'gloss' is not defined

In [53]:
from Preprocessing import *
from PolyphonicPreprocessing import *

Mono = True

TEMPO_MIN, TEMPO_MAX = 60, 200  # Typical tempo range
PROGRAM_MIN, PROGRAM_MAX = 1, 130  # MIDI program range
GENRE_MIN, GENRE_MAX = 0, 9

def NormCond(tempo, programs, genre, Mono):

   tempo_norm = (tempo - TEMPO_MIN) / (TEMPO_MAX - TEMPO_MIN)
    
   if Mono:
      programs_norm = (programs - PROGRAM_MIN) / (PROGRAM_MAX - PROGRAM_MIN)
      return [tempo_norm] + [programs_norm]
   else:
      programs_norm = [(p - PROGRAM_MIN) / (PROGRAM_MAX - PROGRAM_MIN) for p in programs]
      genre_norm = (genre - GENRE_MIN) / (GENRE_MAX - GENRE_MIN)
      return [tempo_norm, genre_norm] + programs_norm
    
   
if Mono:
   Dataset = torch.load('MonophonicDataset.pt')
   
else:
   Dataset = PolyphonicPreProcessing(nDir = 500)

In [54]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('GPU available')
else:
    device = torch.device("cpu")
    print('GPU not available')


def LoadModel(Cond1D_Size, instrumentSize, Which):

    Path = os.path.realpath('ModelParameters')

    files = [
        ['MonoGenParam.torch'],
        ['PolyGenParam.torch']
    ]

    generator = Generator(input_size=100, cond_1d_size=Cond1D_Size, instrument_size=instrumentSize, n_hlayers=128)
    generator.apply(weights_init)
    generator.to(device)
    # Load the state dict previously saved
    generator_state_dict = torch.load(os.path.join(Path, files[Which][0]), map_location=torch.device('cpu'))
    # Update the network parameters
    generator.load_state_dict(generator_state_dict)

    return generator

                                                    #MOnophonic or polyphonic
if Mono: generator = LoadModel(Cond1D_Size=2, instrumentSize=1, Which=0)
else: generator = LoadModel(Cond1D_Size=6, instrumentSize=4, Which=1)

GPU not available


In [68]:
device = torch.device('cpu')

generator.eval().to(device)

bar = np.random.randint(0, 25000)

if Mono:
   Instrument = 'Bass'
   prev_bar = Dataset[Instrument][bar]['Bars'][0].to_dense().float().to(device)
   InstrumentCode = Dataset[Instrument][bar]['Program']
   Tempo = Dataset[Instrument][bar]['Tempo'][0]
   Cond1D_Norm = torch.tensor([NormCond(Tempo, InstrumentCode, 0, Mono)], dtype= torch.float32, device = device)


else:
   prev_bar = Dataset[bar]['Bars'][0].to_dense().float().to(device)
   InstrumentCode = Dataset[bar]['Program']
   Tempo = Dataset[bar]['Tempo']
   Genre = Dataset[bar]['Genre']
   Cond1D = [Tempo, Genre] + InstrumentCode
   #Cond1D = [100, 2, 2, 24, 32, 130]

   Cond1D_Norm = torch.tensor([NormCond(Tempo, InstrumentCode, Genre, Mono)], dtype= torch.float32, device = device)


Bars = []
Bars.append(prev_bar)

if Mono:
   prev_bar = prev_bar.unsqueeze(0).unsqueeze(0) 
else:
   prev_bar = prev_bar.unsqueeze(0)


for i in range(7):
   noise = torch.rand([1, 100], device=device)*0.2

   #print(np.mean(np.array(noise)))

   with torch.no_grad():
      generated_bar = generator(noise, prev_bar, Cond1D_Norm, 1)

   mean = generated_bar.mean().item()
   std = generated_bar.std().item()
   binary_bar = (generated_bar > mean + 4*std).float()
   if Mono: 
      Bars.append(binary_bar.squeeze(0).squeeze(0).cpu().numpy()) 
      prev_bar = Dataset[Instrument][bar + i + 1]['Bars'][0].unsqueeze(0).unsqueeze(0).to_dense().float().to(device)
      #prev_bar = binary_bar.detach()
   else: 
      Bars.append(binary_bar.squeeze(0).cpu().numpy()) 
      #prev_bar = Dataset[bar + i + 1]['Bars'][0].unsqueeze(0).to_dense().float().to(device)
      prev_bar = binary_bar.detach()
   print(mean, std)
   


if Mono:
   ConcBars = np.concatenate(Bars, axis = 1)
   MonoBarsToMIDI(ConcBars, Instrument=InstrumentCode, title='GeneratedAudio/Monotest')

else:
   PolyConcBars = np.concatenate(Bars, axis = 2)
   PolyBarsToMIDI(PolyConcBars, Cond1D=Cond1D, title='GeneratedAudio/Polytest')


print(InstrumentCode)


0.04425717145204544 0.06909490376710892
0.04604472219944 0.07575421780347824
0.04508315399289131 0.07025603950023651
0.04645500332117081 0.058509524911642075
0.04194045811891556 0.06809889525175095
0.046523578464984894 0.05806519463658333
0.04181010276079178 0.06807234138250351
33


In [81]:
np.sum(Bars[3])

44.0

Save networks and optimizers states

In [ ]:
#torch.save(discriminator.state_dict(), 'discriminator_parameters.torch')
torch.save(generator.state_dict(), 'generator_parameters.torch')
print('Saved Model')

#torch.save(dis_opt.state_dict(), 'dis_opt_state.torch')
torch.save(gen_opt.state_dict(), 'gen_opt_state.torch')
print('Saved Optimizer')